In [1]:
%run ToricFunctions.ipynb

In [2]:
###########################################################################################
######################                                               ###################### 
######################                  POLYTOPES                    ######################
######################                                               ###################### 
###########################################################################################

#################################################################
######  Create a polytope from a set of rays and a vector  ######
#################################################################

def poly(rays,A):
    ieq=(matrix(A).transpose()).augment(matrix(rays)).rows()
    return Polyhedron(ieqs=ieq)

##############################################
#### Invariant divisors giving a polytope ####
##############################################

def HrepA(P): 
    return [Hrep.A() for Hrep in P.Hrepresentation()]

############################################################
#### Coefficients of the polarization giving a polytope ####
############################################################

def HrepB(P):
    return vector([Hrep.b() for Hrep in P.Hrepresentation()])    

In [3]:
###########################################################################################
######################                                               ###################### 
######################        C^*-ACTIONS ON A TORIC VARIETY         ######################
######################                                               ###################### 
###########################################################################################

from collections import Counter

###############################################################
####  Arrange the fixed points according to their weights  ####
###############################################################

def sort_fixed_points(P,j=None):
    j = j or P.dimension()
    vertices = P.vertices_list()
    weights = sorted({v[j-1] for v in vertices})
    count = Counter(v[j-1] for v in vertices)
    sorted_fixed_points = [count[w] for w in weights]
    ordered_vertices = sorted(vertices,key=lambda v: v[j-1])
    return weights,sorted_fixed_points,ordered_vertices

###################################################################################
####  Compute weights, criticality and number of fixed points for each weight  ####
###################################################################################

def weights(P,j=None):
    j=j or P.dimension()
    return sort_fixed_points(P,j)[0] 

def criticality(P,j=None):
    j=j or P.dimension()
    return len(sort_fixed_points(P,j)[0])-1 

def fixedpoints(P,j=None):
    j=j or P.dimension()
    return sort_fixed_points(P,j)[1]

############################################################################
#####  Pruning: Add the inequalities  a \le x \le b to the Hrep of P  ######
############################################################################

def pruning(P,j,a,b): 
    # Get the H-representation of P
    H = [vector(list(HrepB(P)[i]) + list(HrepA(P)[i])) 
        for i in range(len(P.Hrepresentation()))]
    # Create the inequalities
    lp = [0] * len(H[0])
    lm = [0] * len(H[0])
    lp[j],lm[j] = 1,-1
    lp[0],lm[0] = -a,b
    # Append the inequalities to the H-representation
    H.append(vector(lp))
    H.append(vector(lm))
    # Return the new polytope with the added inequalities
    return Polyhedron(ieqs=H)
    
###########################################
###   Cut the slice x=a of the polytope ###
###########################################

def quotient(P,j,a): 
    return pruning(P,j,a,a)

################################################################
#####  Compute an intermediate value between two weights  ######
################################################################

def quotient_weights(P,j=None):
    j=j or P.dimension()
    w = weights(P,j)
    return [(w[i] + w[i+1]) / 2 for i in range(len(w) - 1)]

############################################################
#####  Compute the geometric quotients of the action  ######
############################################################

def geometric_quotients(P,j):
    w = quotient_weights(P,j) 
    c = criticality(P,j)
    return [quotient(P,j,w[i]) for i in range(c)]
    
##############################################
##### Check properties of a toric variety ####
##############################################

def check_condition(condition):
    return "is" if condition else "is not" 

def check(X):
    if 'Polyhedra' in str(X.category()):
        X = ToricVariety(NormalFan(X))
    fan=X.fan()
    print("The variety {} complete, {} Q-factorial, {} smooth, {} Fano".format(check_condition(fan.is_complete()), 
        check_condition(fan.is_simplicial()), check_condition(fan.is_smooth()), check_condition(is_Fano(X)))) 

#################################################
#####   Display information on the action  ######
#################################################

def action_info(P,j=None):
    j=j or P.dimension()
    btype=["divisorial contraction","flip","divisorial extraction","non elementary modification"]
    GQ=geometric_quotients(P,j)
    print("The criticality of the action is {}".format(criticality(P,j))) 
    print("The weights are {}".format(weights(P,j)))
    print("The polytopes of fixed point components have", fixedpoints(P,j), "vertices")
    for i in [0..criticality(P,j)-1]:
        print("The quotient GX_{}".format(i), "is a {}-dimensional polytope".format(GQ[i].dimension()),
        "with {} vertices".format(len(GQ[i].vertices())), "and Picard number {}".format(picard_number(GQ[i])))
    for i in [0..criticality(P,j)-2]:    
        print("The map GX_{} --> GX_{}".format(i,i+1), "is a", btype[birational_type(GQ[i],GQ[i+1])+1])
    check(P) 
    print("The Picard number is {}".format(picard_number(P)))

In [4]:
###########################################################################################
######################                                               ######################
######################              BIRATIONAL GEOMETRY              ###################### 
######################                                               ######################
###########################################################################################

###################################################################
###   Check if two divisors are in the same chamber of Eff(Y)   ###
###   Y is a toric variety                                      ###
###   A,B are divisors                                          ###
###################################################################

def same_chamber(Y,A,B):
    r=Y.fan().rays()
    P, Q, PQ = poly(r,A), poly(r,B), poly(r,A+B)
    test1 = PQ.is_combinatorially_isomorphic(P)
    test2 = PQ.is_combinatorially_isomorphic(Q)
    return test1 and test2

#########################################################################################
###   Modify a divisor staying in the same (maximal dimensional!) chamber of Eff(X)   ###
###   X is a toric variety                                                            ###
###   A is a Cartier divisor, given as a vector                                       ###
#########################################################################################

def modify(Y,A):
    r=Y.fan().rays()
    D=vector([randint(-1000,1000) for _ in range(len(r))])
    B=[A+D]
    while not same_chamber(Y,A,B[-1]):
        B.append((2**len(B))*A+D)
    return B[-1]
    
########################################################################################
###   Check if a birational map is a wall crossing                                  ####
###   P,Q are polytopes corresponding to divisors in maximal dimensional chambers   ####
########################################################################################

def birational_type(P,Q):
    a=len(P.facets())-len(Q.facets())
    b=len((P+Q).facets())
    if a==0 and b==len(P.facets())+1:
        return 0 
    elif a==1 and b==len(P.facets()):
        return -1  
    elif a==-1 and b==len(Q.facets()):
        return 1  
    return 2  

def is_SQM(P,Q):
    return birational_type(P,Q)==0

def is_extraction(P,Q):
    return birational_type(P,Q)==1

def is_contraction(P,Q):
    return birational_type(P,Q)==-1

def is_wall_crossing(P,Q):
    return birational_type(P,Q) in [0,1,-1]

In [5]:
###########################################################################################
######################                                               ######################
######################             GEOMETRIC REALIZATIONS            ######################
######################                                               ######################
###########################################################################################

############################################
###   Construct a geometric realization  ###
############################################

def geometric_realization(Y,A,B,l=1):
    H=(B-A)/l
    r=[list(ray) for ray in (Y.fan()).rays()]
    M=Matrix([[0]*len(H),H])
    pr=proj_rays(r,M)
    L=vector(QQ,[1 if i == len(pr)-2 else 0 for i in range(len(pr))])
    pullback_A=A.concatenate(vector([0,0]))
    m=lcm([x.denominator() for x in weights(poly(pr,l*L+pullback_A))])
    return poly(pr,m*(l*L+pullback_A)) 

############################################
###   Check if a realization is sharp    ###
############################################

def is_sharp(G,j):
    r=criticality(G,j)-1
    if r==0:
        return true
    Q=[geometric_quotients(G,j)[i] for i in range(r+1)]
    return all(is_wall_crossing(Q[i],Q[i+1]) for i in range(r))
        
##########################################
###   Construct a sharp realization    ###
##########################################

def sharp_realization(Y,A,B):
    n=0
    GR=[geometric_realization(Y,A,B)]
    D=[A]
    j=(GR[0]).dimension()
    while not is_sharp(GR[n],j):
            n+=1
            D.append(modify(X,D[n-1]))
            GR.append(geometric_realization(X,D[n],B))
    return GR[n]   

In [6]:
###########################################################################################
######################                                               ######################
######################                     UNPRUNING                 ######################
######################                                               ######################
###########################################################################################

#######################
#####  Unpruning ######
#######################

def unpruning(Y,A,B,a,b):
    r=[list(ray) for ray in (Y.fan()).rays()]
    M=matrix([A,B])
    pr=proj_rays(r,M)
    n=len(pr)
    D1=vector(QQ, [1 if i == n - 2 else 0 for i in range(n)])
    D2=vector(QQ, [1 if i == n - 1 else 0 for i in range(n)])
    L=M.row(0).concatenate(vector([0,0]))+D1
    m=lcm([x.denominator() for x in weights(poly(pr,L+a*D1+b*D2))])
    return poly(pr,m*(L+a*D1+b*D2))

###############################
#####  Fano realizations ######
###############################

def compute_m(Y,H):
    l=len(Y.fan().rays())
    m=1
    D=vector([1]*l)+vector(H)
    while not is_ample(Y,D):
        m += 1
        D=vector([m]*l)+vector(H)
    return m

def Fano_realization(Y,H):
    l=len(Y.fan().rays())
    m=compute_m(Y,H)
    K=vector([1]*l)
    return unpruning(Y,m*K-H,m*K,m-1,m)